In [64]:
import torch
import torch.nn as nn
import torchvision
import pandas as pd
import numpy as np
import zipfile
import io
import pandas as pd
from IPython.display import display
from collections import defaultdict

zipped_data_path = "../data/clean_data/class-competition.zip"

dataframes = defaultdict(pd.DataFrame)
with zipfile.ZipFile(zipped_data_path, "r") as zip:
    for filename in zip.namelist():
        if filename.endswith(".csv"):
            with zip.open(filename) as f:
                dataframes.update({ filename : pd.read_csv(io.TextIOWrapper(f)) })

                # Lets take a look at the files
                print(f"FILE: {filename}")
                display(dataframes[filename].info())
                display(dataframes[filename].head())

train_data = dataframes["train.csv"]
test_data  = dataframes["test_public.csv"]

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

FILE: train.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710660 entries, 0 to 1710659
Data columns (total 55 columns):
 #   Column         Dtype  
---  ------         -----  
 0   TRIP_ID        int64  
 1   ORIGIN_CALL    float64
 2   ORIGIN_STAND   float64
 3   TAXI_ID        int64  
 4   MISSING_DATA   bool   
 5   POLYLINE       object 
 6   TRAVEL_TIME    int64  
 7   CALL_TYPE_A    bool   
 8   CALL_TYPE_B    bool   
 9   CALL_TYPE_C    bool   
 10  YEAR_2013      bool   
 11  YEAR_2014      bool   
 12  MONTH_1        bool   
 13  MONTH_2        bool   
 14  MONTH_3        bool   
 15  MONTH_4        bool   
 16  MONTH_5        bool   
 17  MONTH_6        bool   
 18  MONTH_7        bool   
 19  MONTH_8        bool   
 20  MONTH_9        bool   
 21  MONTH_10       bool   
 22  MONTH_11       bool   
 23  MONTH_12       bool   
 24  DAY_OF_WEEK_0  bool   
 25  DAY_OF_WEEK_1  bool   
 26  DAY_OF_WEEK_2  bool   
 27  DAY_OF_WEEK_3  bool   
 28  DAY_OF_WEEK_4  bool   
 29

None

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,MISSING_DATA,POLYLINE,TRAVEL_TIME,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,1372636858620000589,NaN,NaN,20000589,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",330,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,1372637303620000596,NaN,7.0,20000596,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",270,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,1372636951620000320,NaN,NaN,20000320,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",960,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,1372636854620000520,NaN,NaN,20000520,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",630,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,1372637091620000337,NaN,NaN,20000337,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",420,False,False,True,...,False,False,False,False,False,False,False,False,False,False


FILE: test_public.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 54 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TRIP_ID        320 non-null    object 
 1   ORIGIN_CALL    72 non-null     float64
 2   ORIGIN_STAND   123 non-null    float64
 3   TAXI_ID        320 non-null    int64  
 4   MISSING_DATA   320 non-null    bool   
 5   POLYLINE       0 non-null      float64
 6   CALL_TYPE_A    320 non-null    bool   
 7   CALL_TYPE_B    320 non-null    bool   
 8   CALL_TYPE_C    320 non-null    bool   
 9   YEAR_2013      320 non-null    bool   
 10  YEAR_2014      320 non-null    bool   
 11  MONTH_1        320 non-null    bool   
 12  MONTH_2        320 non-null    bool   
 13  MONTH_3        320 non-null    bool   
 14  MONTH_4        320 non-null    bool   
 15  MONTH_5        320 non-null    bool   
 16  MONTH_6        320 non-null    bool   
 17  MONTH_7        320 non-null    b

None

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,MISSING_DATA,POLYLINE,CALL_TYPE_A,CALL_TYPE_B,CALL_TYPE_C,YEAR_2013,...,HOUR_14,HOUR_15,HOUR_16,HOUR_17,HOUR_18,HOUR_19,HOUR_20,HOUR_21,HOUR_22,HOUR_23
0,T1,NaN,15.0,20000542,False,NaN,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
1,T2,NaN,57.0,20000108,False,NaN,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
2,T3,NaN,15.0,20000370,False,NaN,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
3,T4,NaN,53.0,20000492,False,NaN,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
4,T5,NaN,18.0,20000621,False,NaN,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False


cuda:0


In [130]:
from sklearn.model_selection import train_test_split

ALL_FEATURES = ['TRIP_ID', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'MISSING_DATA',
 'POLYLINE', 'CALL_TYPE_A', 'CALL_TYPE_B', 'CALL_TYPE_C', 'YEAR_2013', 'YEAR_2014',
 'MONTH_1', 'MONTH_2', 'MONTH_3', 'MONTH_4', 'MONTH_5', 'MONTH_6',
 'MONTH_7', 'MONTH_8', 'MONTH_9', 'MONTH_10', 'MONTH_11', 'MONTH_12',
 'DAY_OF_WEEK_0', 'DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3',
 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5', 'DAY_OF_WEEK_6', 'HOUR_0', 'HOUR_1',
 'HOUR_2', 'HOUR_3', 'HOUR_4', 'HOUR_5', 'HOUR_6', 'HOUR_7', 'HOUR_8',
 'HOUR_9', 'HOUR_10', 'HOUR_11', 'HOUR_12', 'HOUR_13', 'HOUR_14',
 'HOUR_15', 'HOUR_16', 'HOUR_17', 'HOUR_18', 'HOUR_19', 'HOUR_20',
 'HOUR_21', 'HOUR_22', 'HOUR_23']

# We could totally change this. Utilization of these just probably requires further preprocessing.
ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION = ['TRIP_ID', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 'POLYLINE']

train_data_sample = train_data.sample(frac=0.1, random_state=420) # frac is used to control percentage of train data used
X = train_data_sample.drop("TRAVEL_TIME", axis=1)
X = X.loc[:, ~X.columns.isin(ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION)]
y = train_data_sample["TRAVEL_TIME"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=420)

test_features = test_data.loc[:, ~test_data.columns.isin(ALL_FEATURES_NOT_SUITED_FOR_ESTIMATION)]

In [131]:
def test_prediction_to_csv(y_pred, outfile_name):
    
	output_df = pd.DataFrame(test_data["TRIP_ID"])
	output_df["TRAVEL_TIME"] = y_pred
	output_df.head()

	output_df.to_csv(f'../guesses/{outfile_name}', index=False)

def num_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [132]:
from torch.utils.data import DataLoader, Dataset

class TaxiDataset(Dataset):
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels
        
    def __getitem__(self, index):
        x = self.data[index]
        if self.labels is not None:
            y = self.labels[index]
            return x, y
        return x
    
    def __len__(self):
        return len(self.data)
    
X_train = X_train.astype(float)
X_test = X_test.astype(float)
X_test_public = test_features.astype(float) # This is what we can predict on

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.tolist(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.tolist(), dtype=torch.float32)
X_test_public_tensor = torch.tensor(X_test_public.values, dtype=torch.float32)

train_dataset = TaxiDataset(X_train_tensor, y_train_tensor)
test_dataset = TaxiDataset(X_test_tensor, y_test_tensor)
test_public_dataset = TaxiDataset(X_test_public_tensor)

batch_size = 32

dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
dataloader_pred = DataLoader(test_public_dataset, batch_size=batch_size)

In [133]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(49, 128),
      nn.ReLU(),
      nn.Linear(128, 64),
      nn.ReLU(),
      nn.Linear(64, 1)
    )

  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)
   
mlp = MLP()
if torch.cuda.is_available():
    mlp.cuda()

In [134]:
num_parameters(mlp)

14721

In [135]:
import torch.optim as optim

criterion = nn.MSELoss()

In [139]:
def evaluate(network, dataloader_test):
    test_loss = 0.0
    with torch.no_grad():
        for inputs, labels in dataloader_test:
            inputs, labels = inputs.to(device), labels.to(device)

            # Perform forward pass
            outputs = network(inputs)

            # Reshape labels for warning
            labels = labels.view(-1, 1)
            
            # Compute loss
            loss = criterion(outputs, labels)

            test_loss += loss

    return test_loss.cpu().detach().numpy() / len(dataloader_test)

def predict(network, dataloader_pred):
    network.eval()  # Set the model to evaluation mode
    predictions = []

    with torch.no_grad():
        for inputs in dataloader_pred:
            inputs = inputs.to(device)
	
            # Perform forward pass
            outputs = network(inputs)

            outputs = outputs.float()

            # Append predictions to the list
            predictions.append(outputs.cpu().numpy())

            # Convert the list of predictions to a NumPy array
    predictions = np.concatenate(predictions)
    return predictions

In [140]:
from tqdm import tqdm

def pipeline(network, optimizer, epochs=10):

    # BEGIN TRAINING
    train_losses = []

    for epoch in tqdm(range(epochs)):

        running_loss = 0.0

        for inputs, labels in dataloader_train:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Perform forward pass
            outputs = network(inputs)

            # Reshape labels for warning
            labels = labels.view(-1, 1)
            
            # Compute loss
            loss = criterion(outputs, labels)
            
            # Perform backward pass
            loss.backward()
            
            # Perform optimization
            optimizer.step()

            running_loss += loss.cpu().detach().numpy()
        train_losses.append(running_loss / len(dataloader_train))

    print('Training process has finished.')
    
    # BEGIN TESTING
    test_loss = evaluate(network, dataloader_test)

    # BEGIN PREDICTION
    predictions = predict(network, dataloader_pred)
            
    return train_losses, test_loss, predictions

In [141]:
optimizer = optim.SGD(mlp.parameters(), lr=0.01)
train_losses, test_loss, predictions = pipeline(mlp, optimizer)

100%|██████████| 10/10 [01:07<00:00,  6.79s/it]


Training process has finished.


In [142]:
print(f"Final Train Loss: {train_losses[-1]}")
print(f"Test Loss: {test_loss}")


Final Train Loss: 451030.6449358487
Test Loss: 597429.0542056075


In [143]:
test_prediction_to_csv(predictions, "predicting_three_layer_mlp.csv")